### Crypto Data API pipeline
- Get data from CoinGecko API
- Transform and get the data we need from the whole JSON set
- Load into a db that isn't on port 5432, change in postgresql.conf file where port = 5432 to port = 8000 or sth else


#### 1. Extract data from API

In [ ]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

load_dotenv()
API_KEY = os.getenv("API_KEY")
DB_URL = os.getenv("DB_URL")
TEST_DB_URL = os.getenv("TEST_DB_URL") # port 5005 on students server
#print(TEST_DB_URL)

In [2]:

coin_list = ['cardano', 'polkadot', 'chainlink', 'litecoin', 'uniswap', 'stellar', 'aptos', 'ripple', 'avalanche-2']
name_list = []
price_list = []
cap_list = []
volume_list = []
time_list = []

for coin in coin_list: # loop thru the list of coins and extract its data
    url = f"https://api.coingecko.com/api/v3/coins/{coin}?localization=false&tickers=false&community_data=false&developer_data=false&sparkline=false&dex_pair_format=symbol"

    headers = {
        "accept": "application/json",
        "x-cg-demo-api-key": API_KEY
    }

    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()

        name_list.append(data["name"])
        price_list.append(data["market_data"]['current_price']['usd'])
        cap_list.append(data['market_data']['market_cap']['usd'])
        volume_list.append(data['market_data']['total_volume']['usd'])
        time_list.append(data['market_data']['last_updated'])

        coin_data = {
            'name': name_list,
            'current_price': price_list,
            'market_cap': cap_list,
            'total_volume': volume_list,
            'time': time_list
        }
    else:
        print(f"Requests error: {response.status_code}, {response.text}")

coin_data

{'name': ['Cardano',
  'Polkadot',
  'Chainlink',
  'Litecoin',
  'Uniswap',
  'Stellar',
  'Aptos',
  'XRP',
  'Avalanche'],
 'current_price': [0.697935,
  4.19,
  14.39,
  90.53,
  6.8,
  0.274735,
  4.98,
  2.25,
  21.45],
 'market_cap': [25197582913,
  6380775860,
  9452957167,
  6874421327,
  4087409209,
  8564133643,
  3141318308,
  132044882678,
  9043235093],
 'total_volume': [470370952,
  161377373,
  289883489,
  259733253,
  462490678,
  173830325,
  116282224,
  2066753989,
  328853114],
 'time': ['2025-06-04T09:03:31.651Z',
  '2025-06-04T09:03:31.149Z',
  '2025-06-04T09:03:36.071Z',
  '2025-06-04T09:03:31.083Z',
  '2025-06-04T09:03:34.109Z',
  '2025-06-04T09:03:31.822Z',
  '2025-06-04T09:03:33.703Z',
  '2025-06-04T09:03:39.091Z',
  '2025-06-04T09:03:30.989Z']}

In [3]:
df = pd.DataFrame(coin_data)
df.head()

,name,current_price,market_cap,total_volume,time
0,Cardano,0.697935,25197582913,470370952,2025-06-04T09:03:31.651Z
1,Polkadot,4.190000,6380775860,161377373,2025-06-04T09:03:31.149Z
2,Chainlink,14.390000,9452957167,289883489,2025-06-04T09:03:36.071Z
3,Litecoin,90.530000,6874421327,259733253,2025-06-04T09:03:31.083Z
4,Uniswap,6.800000,4087409209,462490678,2025-06-04T09:03:34.109Z


In [4]:
# let's set time column as a pd.to_datetime obj
df['time'] = pd.to_datetime(df['time'], utc=True)
df.head()

,name,current_price,market_cap,total_volume,time
0,Cardano,0.697935,25197582913,470370952,2025-06-04 09:03:31.651000+00:00
1,Polkadot,4.190000,6380775860,161377373,2025-06-04 09:03:31.149000+00:00
2,Chainlink,14.390000,9452957167,289883489,2025-06-04 09:03:36.071000+00:00
3,Litecoin,90.530000,6874421327,259733253,2025-06-04 09:03:31.083000+00:00
4,Uniswap,6.800000,4087409209,462490678,2025-06-04 09:03:34.109000+00:00


In [5]:
# set the time column as the index for time-series analysis
#df.set_index('time', inplace=True)
df.head()
df['name']

0      Cardano
1     Polkadot
2    Chainlink
3     Litecoin
4      Uniswap
5      Stellar
6        Aptos
7          XRP
8    Avalanche
Name: name, dtype: object

In [6]:
# Transpose the dataframe, setting coin_name as the column for easier querying
df.set_index('time', inplace=True)
df['name']


time
2025-06-04 09:03:31.651000+00:00      Cardano
2025-06-04 09:03:31.149000+00:00     Polkadot
2025-06-04 09:03:36.071000+00:00    Chainlink
2025-06-04 09:03:31.083000+00:00     Litecoin
2025-06-04 09:03:34.109000+00:00      Uniswap
2025-06-04 09:03:31.822000+00:00      Stellar
2025-06-04 09:03:33.703000+00:00        Aptos
2025-06-04 09:03:39.091000+00:00          XRP
2025-06-04 09:03:30.989000+00:00    Avalanche
Name: name, dtype: object

In [7]:
# the dataset is clean, loading it into a mock table in the public schema
engine = create_engine(DB_URL)
try:
    df.to_sql(name='mock_crypto_data', con=engine, schema='public', index=True, if_exists='append')
    print("Data loaded successfully")
except Exception as e:
    print(f"Loading data: {e}")


Data loaded successfully


### Test ETL complete.
 Airflow can now be initialized using this info and hourly DAG for this ETL pipeline be set up.
 Tasks are:
- extract_data: extracting data from API by looping through the list of coin IDs
- transform_data: cleans data by setting time into a pd.to_datetime obj for time series analysis, setting time as the index and checking for column datatypes
- load_to_db: loads the transformed dataframe into a Postgres DB whose port != 5432. Change this in the postgresql.conf file manually.


In [3]:
from datetime import datetime

today = datetime.today().strftime('%Y:%m:%d')
today

'2025:06:04'